In [5]:
BROKERS = ["host.docker.internal:9092"]

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
import json
import threading
import time
import uuid

def produce_messages():
    producer = KafkaProducer(
        bootstrap_servers=BROKERS,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    for i in range(5):
        message = {
            "user_id": i,
            "action": "activate",
            "timestamp": "2025-03-12T12:00:00"
        }
        producer.send('activate-layer', message)
        print("Message sent: ", message)
        time.sleep(1)

    producer.flush()

def consume_messages():
    unique_group_id = f'my-group-{uuid.uuid4()}'
    consumer = KafkaConsumer(
        'activate-layer',
        bootstrap_servers=BROKERS,
        auto_offset_reset='latest',
        enable_auto_commit=True,
        group_id=unique_group_id,
        value_deserializer=lambda x: json.loads(x.decode('utf-8'))
    )

    print("Starting consumer...")
    count = 0
    for message in consumer:
        print("Received message: ", message.value)
        count += 1
        if count >= 5:
            break

if __name__ == "__main__":
    # Start consumer in a separate thread
    consumer_thread = threading.Thread(target=consume_messages)
    consumer_thread.start()

    # Ensure the consumer is fully ready
    time.sleep(2)

    produce_messages()

    consumer_thread.join()


Starting consumer...
Message sent:  {'user_id': 0, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 1, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 2, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 2, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 3, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 3, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 4, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 4, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
